In [ ]:
!pip install -q faiss-cpu sentence-transformers transformers accelerate bitsandbytes

In [1]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import pandas as pd

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')  # MUST match the one used for .npy
index = faiss.read_index('/kaggle/input/cpe595/symptom_faiss.index')

df = pd.read_csv('/kaggle/input/cpe595/Symptoms_Text_Converted.csv')
symptom_texts = df['symptom_text'].tolist()
diagnoses = df['diseases'].tolist()

# -----------------------------
# Retrieval Function
# -----------------------------
def retrieve_symptoms(user_query, top_k=3):
    query_vector = model.encode([user_query]).astype('float32')
    faiss.normalize_L2(query_vector)  # Important for cosine similarity

    distances, indices = index.search(query_vector, top_k)

    results = []
    seen = set()

    for idx in indices[0]:
        symptom = symptom_texts[idx]
        diagnosis = diagnoses[idx]
        if symptom not in seen:
            results.append((symptom, diagnosis))
            seen.add(symptom)
    return results

2025-05-09 05:51:59.426277: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746769919.917050     190 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746769920.054667     190 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
from sklearn.metrics.pairwise import cosine_similarity

def rerank_with_semantics(
    user_query,
    retrieved_results,
    min_similarity=0.5,
    min_symptom_words=3
):
    # Encode query and retrieved symptoms
    query_embedding = model.encode([user_query], convert_to_tensor=True)
    symptom_texts = [symptom for symptom, _ in retrieved_results]
    symptom_embeddings = model.encode(symptom_texts, convert_to_tensor=True)

    # Compute cosine similarity
    similarities = cosine_similarity(
        query_embedding.cpu().numpy(),
        symptom_embeddings.cpu().numpy()
    )[0]

    # Filter by semantic similarity and symptom word count
    scored = [
        (symptom, diagnosis, sim)
        for (symptom, diagnosis), sim in zip(retrieved_results, similarities)
        if sim >= min_similarity and len(symptom.split()) >= min_symptom_words
    ]

    # Sort by similarity score
    scored.sort(key=lambda x: x[2], reverse=True)

    return [(symptom, diagnosis) for symptom, diagnosis, _ in scored]


In [16]:
user_input = "high fever and cough"
raw_matches = retrieve_symptoms(user_input, top_k=15)  # Get enough for reranking
reranked_matches = rerank_with_semantics(user_input, raw_matches, min_similarity=0.5)
final_matches = reranked_matches[:5]

for i, (symptom, diagnosis) in enumerate(final_matches, 1):
    print(f"\nMatch {i}:")
    print(f"Symptom: {symptom}")
    print(f"Diagnosis: {diagnosis}")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Match 1:
Symptom: sore throat cough feeling ill fever
Diagnosis: mononucleosis

Match 2:
Symptom: cough wheezing fever
Diagnosis: asthma

Match 3:
Symptom: cough nasal congestion feeling ill fever
Diagnosis: mononucleosis

Match 4:
Symptom: sore throat cough fever nosebleed
Diagnosis: nose disorder

Match 5:
Symptom: cough fever painful sinuses nosebleed
Diagnosis: nose disorder


In [4]:
def construct_prompt(user_input, final_matches):
    prompt = (
        f"You are a medical assistant. A patient reports: \"{user_input}\".\n"
        f"Analyze the symptoms by comparing them to known cases below.\n"
        f"Use this to make a medically sound diagnosis.\n"
    )
    for i, (symptom, diagnosis) in enumerate(final_matches, 1):
        prompt += f"Case {i}: Symptoms: {symptom} → Diagnosis: {diagnosis}\n"
    prompt += (
        "\nBased on these cases, what could be a likely diagnosis?\n"
        "Write a short, complete explanation (1–3 sentences)."
    )
    return prompt


In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

# Use Mistral-7B-Instruct
model_name = "HuggingFaceH4/zephyr-7b-beta"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model_lm = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"  # auto-places on GPU
)

# Text generation pipeline (use text-generation instead of text2text-generation)
generator = pipeline("text-generation", model=model_lm, tokenizer=tokenizer)

# Use generate_explanation with Mistral
def generate_explanation(user_input, top_k=3):
    matches = retrieve_symptoms(user_input, top_k=top_k+3)
    reranked = rerank_with_semantics(user_input, matches)  # ✅ semantic reranker here
    final_matches = reranked[:top_k]

    prompt = construct_prompt(user_input, final_matches)
    print("\n📜 Prompt Preview:\n", prompt)

    response = generator(
        prompt,
        max_new_tokens=200,
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        top_k=50
    )
    explanation = response[0]['generated_text'].strip()
    return final_matches, explanation

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Device set to use cuda:0


In [18]:
query = "high fever and cough"
results, explanation = generate_explanation(query, top_k=3)

print("\n🔍 Top Matches:")
for i, (s, d) in enumerate(results, 1):
    print(f"{i}. {s} → {d}")

print("\n🧠 Explanation:")
print(explanation)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


📜 Prompt Preview:
 You are a medical assistant. A patient reports: "high fever and cough".
Analyze the symptoms by comparing them to known cases below.
Use this to make a medically sound diagnosis.
Case 1: Symptoms: sore throat cough feeling ill fever → Diagnosis: mononucleosis
Case 2: Symptoms: cough wheezing fever → Diagnosis: asthma
Case 3: Symptoms: cough nasal congestion feeling ill fever → Diagnosis: mononucleosis

Based on these cases, what could be a likely diagnosis?
Write a short, complete explanation (1–3 sentences).

🔍 Top Matches:
1. sore throat cough feeling ill fever → mononucleosis
2. cough wheezing fever → asthma
3. cough nasal congestion feeling ill fever → mononucleosis

🧠 Explanation:
You are a medical assistant. A patient reports: "high fever and cough".
Analyze the symptoms by comparing them to known cases below.
Use this to make a medically sound diagnosis.
Case 1: Symptoms: sore throat cough feeling ill fever → Diagnosis: mononucleosis
Case 2: Symptoms: cough w